## Peer2Peer Loss Given Default Prediction



This solution predicts the Loss due to default of the borrowers who are most likely to default on their Consumer loans in Peer-to-Peer lending. During the training stage, the solution understands the dataset, handles missing data and class imbalance, conducts feature interaction on the training data and selects a subset of best features based on feature importance. It then trains on multiple ML models, identifies the best performing model and tunes it accordingly. This trained model is then selected for prediction on the test data.



This sample notebook shows you how to deploy Peer2Peer Loss Given Default Prediction using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Peer-to-Peer Default Loss Prediction. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
    3. Make sure to install below mentioned python libraries before running this notebook
    
#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Usage Instruction](#2.-Usage-Instruction)
3. [Training](#3.-Training) 
4. [Sample Data](#4.-Sample-Data)
5. [Perform batch inference](#5.-Perform-batch-inference)
6. [Create an endpoint and perform real-time inference](#6.-Create-an-endpoint-and-perform-real-time-inference)
7. [Clean-up](#7.-Clean-up)
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the algorithm listing page **Peer2Peer Loss Given Default Prediction**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### 2. Usage Instruction

The deployed solution has these 2 steps:
1: The solution trains on user provided historical training data and builds & saves a ML model which is a representation of the historical data.
2: Once the model is generated, the solution can be used to predict on any new data as input file. 
To achieve this end, the solution deploys the following 2 APIs over AWS Sagemaker:

1.Training API: The system trains on user provided historical data with relevant features and builds & saves a ML model.
2.Testing API: Once the model is generated, the solution can be used to predict the loss given default for a given test data. 


## Input
Supported content types: `text/csv`
* Following is the mandatory inputs for the APIs:
 * The Target variable should be named**LossGivenDefault**, this features tells us the Percentage of amount lost when Borrower Defaults.
 

Note: 
1.	Two separate csv input files are required for training and testing.
  * Name of the training file should be train.csv
  * Name of the testing file should be sample_input.csv
  
## Output
• Content type: ` text/csv`
* Solution will generate an output csv file which will contain an additional columns as the prediction along with the columns in the new data.
*  The additional columns will be **Label**.

## Invoking endpoint

### AWS CLI Command
If you are using real time inferencing, please create the endpoint first and then use the  following command to invoke it:

``` bash 
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 sample_output.csv
```
Substitute the following parameters:
* `"model-name"` - name of the inference endpoint where the model is deployed
* `file_name` - input csv file name 
* `text/csv` - content type of the given input 
* `sample_output.csv` - filename where the inference results are written to.

In [1]:
import base64 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image
import numpy as np
import pandas as pd
import cv2
from numpy import asarray

In [2]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [3]:
# S3 prefixes
common_prefix = "Peer_to_Peer_Default_Loss_Prediction"
training_input_prefix = common_prefix + "/training-input-data"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"

In [4]:
sagemaker_session = sage.Session()

In [5]:
# Maintaine this directory structure
TRAINING_WORKDIR = "data/training"

TRAINING_DATA = TRAINING_WORKDIR + "/train.csv"

In [6]:
TRAINING_WORKDIR = "data/training"

training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print ("Training Data Location " + training_input)

Training Data Location s3://sagemaker-us-east-2-786796469737/Peer_to_Peer_Default_Loss_Prediction/training-input-data


### 3. Training 

In [7]:
import json
import time
from sagemaker.algorithm import AlgorithmEstimator

##### Algorithm ARN

In [8]:
algorithm_arn ='' # put your ARN here

In [10]:
algo = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    instance_count=1,
    instance_type='ml.m5.12xlarge',
    base_job_name='Peer-to-Peer-Default-Loss-Prediction')

In [11]:
print ("Now run the training job using algorithm arn %s in region %s" % (algorithm_arn, sagemaker_session.boto_region_name))
algo.fit({'training': training_input})

Now run the training job using algorithm arn arn:aws:sagemaker:us-east-2:786796469737:algorithm/default-loss-prediction-v3 in region us-east-2
2021-07-30 11:06:58 Starting - Starting the training job...
2021-07-30 11:07:21 Starting - Launching requested ML instancesProfilerReport-1627643218: InProgress
...
2021-07-30 11:07:48 Starting - Preparing the instances for training......
2021-07-30 11:08:59 Downloading - Downloading input data...
2021-07-30 11:09:22 Training - Downloading the training image...
2021-07-30 11:09:52 Training - Training image download completed. Training in progress..RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
Starting the training.
(68916, 12)
['WorkExperience'] ['Age', 'LoanDuration', 'MonthlyPayment', 'Amount', 'Gender', 'IncomeTotal', 'UseOfLoan', 'Education', 'HomeOwnershipType', 'OccupationArea']
IntProgress(value=0, description='Processing: ', max=3)
                                                                  

                                                                      
                                                                      
Initiated  . . . . . . . . . . . . . . . . . .                11:10:01
Status     . . . . . . . . . . . . . . . . . .         Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Least Angle Regression
                                                                      
                                                                      
Initiated  . . . . . . . . . . . . . . . . . .                11:10:01
Status     . . . . . . . . . . . . . . . . . .         Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Least Angle Regression
                                                                      
                                                                      
Initiated  . . . . . . . . . . . . . . . . . .                11:10:01
Status     . . . . . . . . . . . . . . . . . .     Selecting Estimator
Estima

                                                                       
                                                                       
Initiated  . . . . . . . . . . . . . . . . . .                 11:10:01
Status     . . . . . . . . . . . . . . . . . .          Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Decision Tree Regressor
                                                                       
                                                                       
Initiated  . . . . . . . . . . . . . . . . . .                 11:10:01
Status     . . . . . . . . . . . . . . . . . .          Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Decision Tree Regressor
                                                                       
                                                                       
Initiated  . . . . . . . . . . . . . . . . . .                 11:10:01
Status     . . . . . . . . . . . . . . . . . .      Selecting Es

                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     11:10:01
Status     . . . . . . . . . . . . . . . . . .              Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Gradient Boosting Regressor
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     11:10:01
Status     . . . . . . . . . . . . . . . . . .              Fitting 3 Folds
Estimator  . . . . . . . . . . . . . . . . . .  Gradient Boosting Regressor
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     11:10:01
Status     .


2021-07-30 11:10:55 Uploading - Uploading generated training model                                    Model         MAE  ...        MAPE  TT (Sec)
rf                Random Forest Regressor      0.1228  ...      0.1976    1.2933
lightgbm  Light Gradient Boosting Machine      0.1384  ...      0.2171    0.2567
et                  Extra Trees Regressor      0.1273  ...      0.2049    0.8067
gbr           Gradient Boosting Regressor      0.1674  ...      0.2541    3.3633
knn                 K Neighbors Regressor      0.1633  ...      0.2646    0.7567
dt                Decision Tree Regressor      0.1348  ...      0.2454    0.4933
ridge                    Ridge Regression      0.2217  ...      0.3222    0.3467
br                         Bayesian Ridge      0.2217  ...      0.3219    0.3733
omp           Orthogonal Matching Pursuit      0.2229  ...      0.3230    0.3500
lr                      Linear Regression      0.2243  ...      0.3240    0.5033
ada                    AdaBoost Regressor 

### 4. Sample Data

In [12]:
# maintaine this directory structure
import os
TRANSFORM_WORKDIR = "data/transform"
sample_data = pd.read_csv(TRANSFORM_WORKDIR + "/sample_input.csv",index_col=0)
sample_data.head(10)

,LoanDuration,MonthlyPayment,Amount,Gender,IncomeTotal,UseOfLoan,Education,WorkExperience,HomeOwnershipType,OccupationArea,LossGivenDefault
Age,,,,,,,,,,,
53,60,62.05,2125,1,354,2,4,15To25Years,1,1,0.58
50,60,84.75,3000,1,900,3,5,MoreThan25Years,1,7,0.65
42,60,63.53,1500,0,863,2,2,5To10Years,2,1,0.90
34,48,101.19,1090,1,697,7,4,5To10Years,3,1,0.75
31,60,243.19,775,1,970,7,4,10To15Years,4,7,0.90
22,60,155.89,3190,1,430,7,3,LessThan2Years,2,3,0.58
23,60,93.22,2500,1,554,6,4,2To5Years,7,3,0.65
45,60,393.93,465,0,2500,2,5,15To25Years,1,17,0.90
28,60,160.68,3500,0,418,3,2,5To10Years,2,19,0.65


### 5. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [13]:
TRANSFORM_WORKDIR = "data/transform"
transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix) + "/sample_input.csv"
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/Peer_to_Peer_Default_Loss_Prediction/batch-inference-input-data/sample_input.csv


In [14]:
transformer = algo.transformer(1, 'ml.m5.12xlarge')
transformer.transform(transform_input, content_type='text/csv')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

..........
............................RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
Starting the inference server with 48 workers.
[2021-07-30 11:18:54 +0000] [65] [INFO] Starting gunicorn 20.1.0
[2021-07-30 11:18:54 +0000] [65] [INFO] Listening at: unix:/tmp/gunicorn.sock (65)
[2021-07-30 11:18:54 +0000] [65] [INFO] Using worker: gevent
[2021-07-30 11:18:54 +0000] [68] [INFO] Booting worker with pid: 68
[2021-07-30 11:18:54 +0000] [69] [INFO] Booting worker with pid: 69
[2021-07-30 11:18:54 +0000] [117] [INFO] Booting worker with pid: 117
[2021-07-30 11:18:54 +0000] [118] [INFO] Booting worker with pid: 118
[2021-07-30 11:18:54 +0000] [166] [INFO] Booting worker with pid: 166
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
[2021-07-30 11:18:55 +0000] [175] [INFO] Booting worker with pid: 175
[2021-07-30 11:18:55 +0000] [223] [INFO] Booting worker with pid: 223
RuntimeError: module compiled against API ver


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
Starting the inference server with 48 workers.
[2021-07-30 11:18:54 +0000] [65] [INFO] Starting gunicorn 20.1.0
[2021-07-30 11:18:54 +0000] [65] [INFO] Listening at: unix:/tmp/gunicorn.sock (65)
[2021-07-30 11:18:54 +0000] [65] [INFO] Using worker: gevent
[2021-07-30 11:18:54 +0000] [68] [INFO] Booting worker with pid: 68
[2021-07-30 11:18:54 +0000] [69] [INFO] Booting worker with pid: 69
[2021-07-30 11:18:54 +0000] [117] [INFO] Booting worker with pid: 117
[2021-07-30 11:18:54 +0000] [118] [INFO] Booting worker with pid: 118
[2021-07-30 11:18:54 +0000] [166] [INFO] Booting worker with pid: 166
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
[2021-07-30 11:18:55 +0000] [175] [INFO] Booting worker with pid: 175
[2021-07-30 11:18:55 +0000] [223] [INFO] Booting worker with pid: 223
RuntimeError: module compiled against API version 0xe but this version of numpy is 

#### Inspect the Batch Transform Output in S3

In [15]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], 'sample_input.csv')

s3_client = sagemaker_session.boto_session.client('s3')

response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')

In [16]:
bucketFolder = transformer.output_path.rsplit('/')[3]

In [17]:
import boto3
s3_conn = boto3.client("s3")
bucket_name=sagemaker_session.default_bucket()
with open('sample_output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/' + 'sample_input.csv' +'.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


### output

In [18]:
#OUTPUT_WORKDIR = "data/output"

import pandas as pd

out_csv = pd.read_csv("sample_output.csv",index_col=0)
out_csv.head(10)

,LoanDuration,MonthlyPayment,Amount,Gender,IncomeTotal,UseOfLoan,Education,WorkExperience,HomeOwnershipType,OccupationArea,LossGivenDefault,Label
Age,,,,,,,,,,,,
53,60,62.05,2125,1,354,2,4,15To25Years,1,1,0.58,0.586300
50,60,84.75,3000,1,900,3,5,MoreThan25Years,1,7,0.65,0.660300
42,60,63.53,1500,0,863,2,2,5To10Years,2,1,0.90,0.782123
34,48,101.19,1090,1,697,7,4,5To10Years,3,1,0.75,0.754600
31,60,243.19,775,1,970,7,4,10To15Years,4,7,0.90,0.858900
22,60,155.89,3190,1,430,7,3,LessThan2Years,2,3,0.58,0.612500
23,60,93.22,2500,1,554,6,4,2To5Years,7,3,0.65,0.651100
45,60,393.93,465,0,2500,2,5,15To25Years,1,17,0.90,0.851200
28,60,160.68,3500,0,418,3,2,5To10Years,2,19,0.65,0.635500


### 6. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [19]:
model_name='Peer-to-Peer-Default-Loss-Prediction'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.12xlarge'
batch_transform_inference_instance_type='ml.m5.12xlarge'

##### Algorithm ARN

In [20]:
algorithm_arn ='' # put your ARN here

#### A. Create an endpoint

In [21]:
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=algorithm_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = algo.deploy(1, 'ml.m5.12xlarge',endpoint_name=model_name)

..........
-----------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [22]:
import pandas as pd
file_name = 'data/transform/sample_input.csv'
data_frame = pd.read_csv(file_name)

#### C. Perform real-time inference

In [23]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'Peer-to-Peer-Default-Loss-Prediction' \
    --body fileb://$file_name \
    --content-type 'text/csv' \
    --region us-east-2 \
    sample_output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [25]:
out_csv = pd.read_csv("sample_output.csv",index_col=0)
out_csv.head(10)

,LoanDuration,MonthlyPayment,Amount,Gender,IncomeTotal,UseOfLoan,Education,WorkExperience,HomeOwnershipType,OccupationArea,LossGivenDefault,Label
Age,,,,,,,,,,,,
53,60,62.05,2125,1,354,2,4,15To25Years,1,1,0.58,0.586300
50,60,84.75,3000,1,900,3,5,MoreThan25Years,1,7,0.65,0.660300
42,60,63.53,1500,0,863,2,2,5To10Years,2,1,0.90,0.782123
34,48,101.19,1090,1,697,7,4,5To10Years,3,1,0.75,0.754600
31,60,243.19,775,1,970,7,4,10To15Years,4,7,0.90,0.858900
22,60,155.89,3190,1,430,7,3,LessThan2Years,2,3,0.58,0.612500
23,60,93.22,2500,1,554,6,4,2To5Years,7,3,0.65,0.651100
45,60,393.93,465,0,2500,2,5,15To25Years,1,17,0.90,0.851200
28,60,160.68,3500,0,418,3,2,5To10Years,2,19,0.65,0.635500


### 7. Clean-up

#### A. Delete the model

In [26]:
predictor.delete_endpoint()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

